In [7]:
import numpy as np
from comet_ml import Optimizer
from unet import *
from keras import Input
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
import math

C:\Anaconda\envs\py36gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
def build_model():
    input_tensor=Input((256,256,1))
    model=unet(input_tensor,maxpool=False)
    return model

In [2]:
def create_data():    
    #create training data.
    X=np.random.randn(100,256,256,1)
    #generate reference data (label/true data)
    Y=0.8*X+2
    print(X.shape,Y.shape)
    #generate some validation data
    X_val=np.random.randn(100,256,256,1)
    Y_val=0.8*X_val+2
    
    print(X_val.shape,Y_val.shape)
    return (X,Y,X_val,Y_val)

In [3]:
def train(X_train=None,Y_train=None,validation_data=(None,None),batch=None,epochs=None):
    model=build_model()
    model.compile(optimizer=Adam(), loss="mse", metrics=["accuracy"])
    results=model.fit(X_train,Y_train,batch_size=batch,epochs=epochs,validation_data=validation_data)
    return results

In [ ]:
def train_generator(train_gen, train_steps_per_epoch=None,validation_data=(None,None), val_steps_per_epoch=None,epochs=None):
    model=build_model()
    model.compile(optimizer=Adam(), loss="mse", metrics=["accuracy"])
    results=model.fit(train_gen,
                                steps_per_epoch=train_steps_per_epoch,
                                epochs=epochs,
                                validation_data=validation_data,
                                validation_steps=val_steps_per_epoch
                                )
    return results

In [ ]:
BATCH_SIZE=64

(X,Y,X_val,Y_val)=create_data()
#https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch

train_dataset=tf.data.Dataset.from_tensor_slices((X,Y))
val_dataset=tf.data.Dataset.from_tensor_slices((X_val,Y_val))

train_dataset=train_dataset.batch(BATCH_SIZE)
val_dataset=val_dataset.batch(BATCH_SIZE)

train_iter=train_dataset.as_numpy_iterator()
val_iter=val_dataset.as_numpy_iterator()

def data_gen(dataset):
    iter_ds=dataset.as_numpy_iterator()
    #x,y=next(iter_ds)
    yield next(iter_ds)

In [ ]:
for i in range(10):
    xy=data_gen(train_dataset)
    print(i,":",list(xy))